# 14장. 반복형, 반복자, 제너레이터

메모리에 다 들어가지 않는 데이터셋을 검색할 때는 한번에 하나씩 필요할 때 가져오도록 구현해야 하며, 이것이 반복자 패턴이 하는 일이다.

파이썬의 컬렉션은 모두 반복형이며, 다음과 같은 연산을 지원하기 위해 내부적으로 반복자를 사용한다.

* for 루프
* 컬렉션형 생성과 확장
* 텍스트 파일을 한 줄씩 반복
* 지능형 리스트/딕셔너리/집합
* 튜플 언패킹
* 함수 호출 시 *를 이용해서 실제 매개변수를 언패킹

이 장에서는 다음과 같은 내용을 다룬다.

* 반복형 객체를 처리하기 위해 내부적으로 `iter()` 내장 함수를 사용하는 방법
* 파이썬에서 고전적인 반복자 패턴을 구현하는 방법
* 제너레이터가 작동하는 방식을 한 줄씩 상세히 설명
* 고전적인 반복자를 제너레이터 함수나 제너레이터 표현식으로 바꾸는 방법
* 표준 라이브러리에서의 범용 제너레이터 함수의 활용
* 제너레이터를 결합하기 위해 새로 추가된 `yield from`을 사용하는 방법
* 사례 연구: 대형 데이터셋을 사용하도록 설계된 데이터베이스 변환 유틸리티에서 제너레이터 함수 이용
* 비슷해 보이는 제너레이터와 코루틴이 실제로는 아주 다르며, 혼합해서 사용하면 안 되는 이유

## 14.1 Sentence 버전 #1: 단어 시퀀스

먼저 Sentence라는 클래스를 구현하면서 반복형을 알아보자.

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 중복되지 않은 매칭 문자열의 리스트 반환
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성

In [5]:
s = Sentence('"The time has come," the Walrus said,')
s # reprlib.repr() 이 생성한 결과

Sentence('"The time ha... Walrus said,')

In [6]:
for word in s: # Sentence 객체는 반복 가능
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [8]:
s[0], s[5], s[-1] # 인덱스 이용 가능

('The', 'Walrus', 'said')

### 14.1.1 Sequence가 반복 가능한 이유: iter() 함수

파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 `iter(x)`를 자동으로 호출한다. `iter()` 내장 함수는 다음 과정을 수행한다.

1. 객체가 `__iter__()` 메서드를 구현하는지 확인하고, 이 메서드를 호출하여 반복자를 가져옴
1. `__iter__()` 메서드가 없고 `__getitem__()`이 있으면 파이썬은 인덱스 0에서부터 시작해서 항목을 순서대로 가져오는 반복자를 생성
1. 이 과정이 모두 실패하면 파이썬은 `TypeError` 오류를 발생시킴

따라서 모든 파이썬 시퀀스는 `__getitem__()`을 구현하고 있으므로 반복할 수 있다(덕 타이핑!). 

구스 타이핑을 이용하면 반복형에 대한 정의가 단순해질 수 있지만, 융퉁성이 떨어진다. 사실 객체를 반복하기 전에 반복형인지 검사하는 건 불필요한데, 이는 바로 `TypeError`를 발생시키기 때문이다. 따라서 `try/except` 블록으로 처리하는 것이 좋다.

## 14.2 반복형과 반복자

* 반복형 : `iter()` 내장 함수가 반복자를 가져올 수 있는 모든 객체(ex:시퀀스)와 반복자를 반환하는 `__iter__()` 메서드를 구현하는 객체를 반복형이라고 한다.
* 반복자 : 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 `__next__()` 메서드를 구현하는 객체. 파이썬 반복자는 `__iter__()` 메서드도 구현하므로 반복형이기도 하다.

파이썬은 반복형 객체에서 반복자를 가져온다. 다음 예제에서 'ABC'는 반복형이다. 반복자가 보이지 않지만 내부 어딘가에 존재한다.

In [9]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


for 문 대신 while 문을 이용하면 다음과 같다.

In [11]:
s = 'ABC'
it = iter(s) # 반복형에서 반복자 it를 생성
while True:
    try:
        print(next(it))
    except StopIteration: # 더이상 항목이 없으면 반복자가 StopIteration 예외 발생시킴
        del it # 반복자 객체 제거
        break

A
B
C


반복자에 대한 표준 인터페이스는 다음과 같은 메서드 두 개를 정의한다.

* `__next__()` : 다음에 사용할 항목 반환. 항목이 없으면 StopIteration 발생
* `__iter__()` : self를 반환.

반복자는 `next()`를 호출하고 StopIteration 예외를 잡는 방법 외에는 항목이 소진되었는지 확인할 방법이 없다. 그리고 반복자는 재설정할 수 없으며, 반복형에 `iter()`를 호출해서 다시 반복자를 생성해야 한다.

In [12]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
it

In [13]:
next(it)

'Pig'

In [14]:
next(it)

'and'

In [15]:
next(it)

'Pepper'

In [16]:
next(it)

StopIteration: 

In [17]:
list(it)

[]

In [18]:
list(iter(s3))

['Pig', 'and', 'Pepper']

## 14.3 Sentence 버전 #2: 고전적인 반복자

이번에는 고전적인 반복자 패턴에 맞춰 Sentence 클래스를 구현한다. `__iter__()` 특별 메서드를 구현하고 이 메서드가 SentenceIterator를 반환하도록 만든다.

In [19]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 중복되지 않은 매칭 문자열의 리스트 반환
            
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성
    
    def __iter__(self): # __getitem__() 은 구현하지 않아도 __iter__()가 있으므로 반복형
        return SentenceIterator(self.words)
    
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError: # 단어가 없으면 예외 발생
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self
            

사실 SentenceIterator에서 `__iter__()`를 구현할 필요는 없다. 그렇지만 반복자는 `__next__()`와 `__iter__()` 메서드를 모두 구현해야 `issubclass(SentenceIterator, abc.Iterator)` 테스트를 통과할 수 있다.

### 14.3.1 Sentence를 반복자로 만들기: 좋지 않은 생각

반복형과 반복자를 만들 때 흔히 하는 오류는 둘을 혼동하기 때문에 발생한다. 반복형은 호출될 때마다 반복자를 새로 생성하는 `__iter__()` 메서드를 가지고 있으며, 반복자는 개별 항목을 반환하는 `__next__()` 메서드와 `__iter__()` 메서드를 가지고 있다. 따라서 반복자는 반복형이지만 반복형은 반복자가 아니다.

반복자 패턴은 다음과 같은 용도에 사용해야 한다.

* 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우
* 집합 객체의 다중 반복을 지원하는 경우
* 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우


## 14.4 Sentence 버전 #3: 제너레이터 함수

동일한 기능을 파이썬스럽게 구현하려면 SentenceIterator 클래스 대신 제너레이터 함수를 사용한다.

In [20]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 중복되지 않은 매칭 문자열의 리스트 반환
            
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성
    
    def __iter__(self): 
        for word in self.words:
            yield word # 현재 단어를 생성
        return
    # 별도의 반복자 클래스가 필요 없다

### 14.4.1 제너레이터 함수의 작동 방식

본체 안에 `yield` 키워드를 가진 함수는 모두 제너레이터 함수다. 제너레이터 함수는 호출되면 제너레이터 객체를 반환한다. 제너레이터의 작동을 보여주는 간단한 예는 다음과 같다.

In [21]:
def gen_123():
    yield 1
    yield 2
    yield 3
    
gen_123

<function __main__.gen_123()>

In [22]:
gen_123()

<generator object gen_123 at 0x7f2abf13e9a8>

In [23]:
for i in gen_123():
    print(i)

1
2
3


In [24]:
g = gen_123()
next(g)

1

In [25]:
next(g), next(g)

(2, 3)

In [26]:
next(g)

StopIteration: 

제너레이터 함수는 함수 본체를 포함하는 제너레이터 객체를 생성한다. 제너레이터 함수 안에 있는 return 문은 제너레이터 객체가 StopIteration 예외를 발생하게 만든다.

In [27]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.') # 이후 StopIteration 예외 발생
    
for c in gen_AB(): # for문은 next()를 호출
    # for 루프는 g = iter(gen_AB())를 실행 후 next(g)를 호출
    print('-->', c)

start
--> A
continue
--> B
end.


두번째 Sentence 버전은 첫 번째 버전보다 훨씬 짧지만, 더 느긋한 구현이 가능하다.

## 14.5 Sentence 버전 #4: 느긋한 구현

Iterator 인터페이스는 느긋하게 처리하도록 설계되어 있다. `next(my_iterator)`는 한 번에 한 항목만 생성한다.

그동안 구현한 Sentence는 느긋한 버전이 아니다. `__init__()`에서 텍스트 안에 있는 단어들의 리스트를 조급하게 생성해서 `self.words`에 바인딩하기 때문이다. 이것을 느긋하게 구현할 수 있을까?

`re.findter()` 함수는 `re.findall()`의 느긋한 버전으로, 리스트 대신 필요에 따라 `re.Match` 객체를 생성하는 제너레이터를 반환한다. 매칭되는 항목이 많으면 `re.finditer()`가 메모리를 많이 절약해준다.

In [28]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
            
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성
    
    def __iter__(self): 
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 14.6 Sentence 버전 #5: 제너레이터 표현식

앞 예제에서 구현한 Sentence는 제너레이터 표현식으로 바꿀 수 있다.

In [29]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.') 

In [34]:
res1 = [x*3 for x in gen_AB()] # 지능형 리스트는 조급하다

start
continue
end.


In [31]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [32]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x7f2abf13ec00>

In [35]:
for i in res2:
    print('-->', i) # for 루프가 res2를 반복해야 gen_AB()의 본체가 비로소 실행된다.

제너레이터 표현식은 제너레이터를 생성하고, 제너레이터 표현식을 사용하면 Sentence 클래스의 코드를 더 짧게 만들 수 있다.

In [36]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
            
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 14.7 제너레이터 표현식: 언제 사용하나?

제너레이터 표현식은 편리하지만, 제너레이터 표현식이 여러 줄에 걸쳐 있을 때는 가독성을 위해 제너레이터 함수를 사용한다. 또한 제너레이터 함수는 재사용할 수도 있다.

## 14.8 또 다른 예제: 등차수열 제너레이터

특정 자료형의 숫자로 구성된 유한 등차수열을 생성하려면 어떻게 해야 할까?

In [37]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None 이면 무한수열이다.
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin) # step이 float인 경우 등을 대비하기 위함
        forever = self.end is None
        index = 0
        while forever or result < self.end: # 이 루프를 빠져나가면 함수도 빠져나가게 된다.
            yield result
            index += 1
            result = self.begin + self.step * index # 실수로 작업할 때의 오차 누적을 줄이기 위함

ArithmeticProgression 클래스는 range() 함수와 비슷하지만, 시그니처가 `(begin, step[, end])`로, range()의 `(start, stop[, step])`과 차이가 있다.

In [38]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [39]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [41]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [42]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1,3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [45]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

생성된 자료들의 자료형이 begin이나 step의 자료형에 따름에 주의하자.

그러나 이 클래스의 목적이 `__iter__()`를 구현함으로써 제너레이터를 생성하는 것이었다면 클래스를 단지 하나의 제너레이터 함수로 만들어도 충분하다.

In [46]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [47]:
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

In [48]:
from fractions import Fraction
ap = aritprog_gen(0, Fraction(1,3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [49]:
from decimal import Decimal
ap = aritprog_gen(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

위 코드는 상당히 멋있지만, 표준 라이브러리에는 바로 사용할 수 있는 제너레이터가 아주 많다. 

### 14.8.1 itertools를 이용한 등차수열

파이썬 3.4의 itertools 모듈에는 제너레이터 함수가 19개 있다. 예를 들어 `itertools.count()` 함수는 숫자를 생성하는 제너레이터를 반환한다.

In [2]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [3]:
next(gen), next(gen), next(gen), next(gen)

(1.5, 2.0, 2.5, 3.0)

그러나 `itertools.count()`는 끝이 없으므로 `list(count()`를 시도하면 파이썬 인터프리터는 가용 메모리보다 더 큰 리스트를 만들려고 시도하면서 잠시 후 실패한다.

`itertools.takewhile()`이라는 함수는 다른 제너레이터를 소비하면서 주어진 조건식이 False가 되면 중단되는 제너레이터를 생성한다. 이 두 개의 제너레이터를 결합하여 다음과 같이 구현할 수 있다.

In [52]:
gen = itertools.takewhile(lambda n: n<3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

이 두 함수를 이용하여 `aritprog_gen()`을 짧고 멋지게 구현해보자.

In [53]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(fisrt, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

위 함수는 yield 문이 없으므로 제너레이터 함수가 아니다. 그러나 제너레이터를 반환하므로 일종의 제너레이터 팩토리처럼 작동한다.

다음엔 바로 사용할 수 있는 표준 라이브러리의 여러 제너레이터 함수를 살펴본다.

## 14.9 표준 라이브러리의 제너레이터 함수

표준 라이브러리의 itertools나 functools 모듈에서 제공하는 제너레이터 함수들을 살펴보자.

### 필터링 제너레이터 함수들

* itertools.compress(it, seletor_it)
* itertools.dropwhile(predicate, it)
* filter(predicate, it)
* itertools.filterfalse(predicate, it)
* itertools.islice(it, [start,] stop [, step=1])
* itertools.takewhile(predicate, it)

In [54]:
def vowel(c):
    return c.lower() in 'aeiou'

In [57]:
list(filter(vowel, 'Aardvark')) # 참된 값만 반환

['A', 'a', 'a']

In [58]:
list(itertools.filterfalse(vowel, 'Aardvark')) # 거짓값만 반환

['r', 'd', 'v', 'r', 'k']

In [60]:
list(itertools.dropwhile(vowel, 'Aardvark')) # 처음으로 거짓인 게 나오면 이후 반환

['r', 'd', 'v', 'a', 'r', 'k']

In [62]:
list(itertools.takewhile(vowel, 'Aardvark')) # 거짓인 게 나올 때까지만 반환

['A', 'a']

In [63]:
list(itertools.compress('Aardvark', (1,0,1,1,0,1))) # 두번째 인수가 참일 때만 첫번째 인수 항목 반환

['A', 'r', 'd', 'a']

In [67]:
list(itertools.islice('Aardvark', 4)) # s[:stop]를 느긋하게 반환

['A', 'a', 'r', 'd']

In [69]:
list(itertools.islice('Aardvark', 4, 7)) # s[start:stop]를 느긋하게 반환

['v', 'a', 'r']

In [71]:
list(itertools.islice('Aardvark', 1, 7, 2)) # s[start:stop:step]를 느긋하게 반환

['a', 'd', 'a']

### 매핑 제너레이터 함수들

## 14.10 파이썬 3.3의 새로운 구문: yield from

다른 제너레이터에서 생성된 값을 상위 제너레이터 함수가 생성할 때는 중첩된 for 루프를 사용한다. 다음 코드는 제너레이터를 연결하는 코드를 수작업으로 구현한 것이다.

In [84]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [85]:
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

`chain()` 제너레이터 함수는 다음과 같이 구현할 수도 있다. 

In [86]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [87]:
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

`yield from i` 문이 내부 for 루프를 대체한다. 이 구문은 16장에서 다시 살펴본다.

## 14.11 반복형을 리듀스하는 함수

다음 함수들은 모두 반복형을 입력받아 하나의 값을 반환한다. `all()`과 `any()`는 단락 평가 함수로서 결과가 확정되는 즉시 반복자 소비를 중단한다.

* all(it)
* any(it)
* max(it, [key=,] [default=])
* min(it, [key=,] [default=])
* functools.reduce(func, it, [initial])
* sum(it, start=0)

In [73]:
all([1,2,3]), all([1,0,3]), all([]) # 모두 참이면 True, 아니면 False

(True, False, True)

In [74]:
any([1,2,3]), any([1,0,3]), any([0,0,0]), any([]) # 하나라도 참이면 True, 아니면 False

(True, True, False, False)

In [76]:
g = (n for n in [0, 0.0, 7, 8])
any(g), next(g) # True 가 반환되면 멈추므로 8을 반환

(True, 8)

## 14.12 iter() 함수 들여다보기

파이써는 객체 x를 반복해야 할 때 `iter(x)`를 호출한다. 그러나 이 함수는 두 개의 인수를 전달해서 호출할 수도 있다. 첫번째 인수는 값을 생성하기 위해 인수 없이 반복적으로 호출되는 콜러블이며 두번째 인수는 구분 표시로서 콜러블에서 이 값이 반환되면 반복자가 StopIteration 예외를 발생시키도록 만든다.

In [6]:
from random import randint
def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1) # d6을 1이 나올 때까지 굴림
d6_iter

In [7]:
for roll in d6_iter:
    print(roll)

여기서 iter() 함수가 callable_iterator 객체를 반환함에 주의하자. 반복자와 마찬가지로 d6_iter 객체는 일단 소모하고 난 후에는 쓸모가 없어지므로 다시 시작하려면 iter() 함수를 한번 더 호출해서 반복자를 만들어야 한다.

다음 코드는 파일에서 빈 줄을 발견하거나 파일의 끝에 도달할 떄까지 한 줄 씩 읽어서 처리한다.

In [83]:
with open('mydata.txt') as fp:
    for line in iter(fp.readline, ''):
        process_line(line)

FileNotFoundError: [Errno 2] No such file or directory: 'mydata.txt'

끗!